In [1]:
] activate ../ 

In [2]:
using Revise
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using GridInterpolations

using POMDPPolicies

using JLD2
using FileIO
using Reel
using Random
using POMDPs
using POMDPModelTools
using LinearAlgebra

using PedestrianAvoidancePOMDP
using EmergencyBrakingSystem

┌ Info: Recompiling stale cache file /home/xubuntu/.julia/compiled/v1.0/PedestrianAvoidancePOMDP/UuJ2S.ji for PedestrianAvoidancePOMDP [a80ff0fe-e6a0-11e8-3b05-15413bd5ea7a]
└ @ Base loading.jl:1187
┌ Warning: Package PedestrianAvoidancePOMDP does not have Random in its dependencies:
│ - If you have PedestrianAvoidancePOMDP checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PedestrianAvoidancePOMDP
└ Loading Random into PedestrianAvoidancePOMDP from project dependency, future warnings for PedestrianAvoidancePOMDP are suppressed.
┌ Warning: Package EmergencyBrakingSystem does not have Random in its dependencies:
│ - If you have EmergencyBrakingSystem checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need t

In [10]:


algorithm = "EmergencyBrakingSystem"
#algorithm = "PedestrianAvoidancePOMDP"
algorithm = "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"

policy = load("../policy/policy.jld2")["policy"];

# Definition ego vehicle and pedestrian behavior

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0., 10., 20., 30., 40., 50.]


ego_v = vut_speeds[9]
hit_point = hit_points[2]
scenario = scenarios[1]

ego_vehicle = []
ego_a = [] 
collision = []
belief = []
action_pomdp = []
collision_rate = []
ttc = []
risk = []
emergency_brake_request = []
prediction_obstacle = []

### Simulate scenario with parameters above defined
# generate scenario based on scenario type
(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = PedestrianAvoidancePOMDP.generate_scenario(scenario, ego_v, hit_point)
# simulate scenario
if (algorithm == "EmergencyBrakingSystem")
    (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
else
    (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy, algorithm)
end
#evaluate result
(collision2, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = PedestrianAvoidancePOMDP.evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision)



PedestrianAvoidancePOMDP_EmergencyBrakingSystem


MethodError: MethodError: no method matching SparseCat{Array{SingleOCFState,1},Array{Float64,1}}()
Closest candidates are:
  SparseCat{Array{SingleOCFState,1},Array{Float64,1}}(!Matched::Any, !Matched::Any) where {V, P} at /home/xubuntu/.julia/packages/POMDPModelTools/zVVz4/src/distributions/sparse_cat.jl:11

In [9]:
# Visualize scenario
if (algorithm == "EmergencyBrakingSystem")
    duration, fps, render_hist = EmergencyBrakingSystem.animate_record(rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, emergency_brake_request, prediction_obstacle, StaticCamera(VecE2(38.0,0.0),25.0))
else
    duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, prediction_obstacle, CarFollowCamera(1,10.0))
end

film = roll(render_hist, fps = fps, duration = duration)
#write("file.mp4", film) # Write to a webm video


Frames{MIME{Symbol("image/png")}}("/tmp/tmpHKgAku", 0x0000000000000050, 20.0, nothing)

In [12]:



# Evaluate EuroNCAP scenarios

using CSV
using DataFrames


algorithm = "EmergancyBrakingSystem"
#algorithm = "PedestrianAvoidancePOMDP"
#algorithm = "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"

policy_name = "no"
#policy_name = "longitudinal_support"
#policy_name = "longitudinal_lateral_support"
policy = load("../policy/policy.jld2")["policy"];


log_filename = string("results_", algorithm, "_", policy_name, ".csv")

println(log_filename)

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA"]
vut_speeds = [50.] #[10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0.] #[0., 10., 20., 30., 40., 50.]


println("Algorithm to evaluate: ", algorithm, " Policy: ", policy_name, " (if specified)")

# Evaluate all EuroNCAP scenarios
rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, emergency_brake_request, prediction_obstacle, collision, ego_a
results = Vector[]
for scenario in scenarios
    for hit_point in hit_points
        for vut_speed in vut_speeds
            ego_v = vut_speed
            (ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = PedestrianAvoidancePOMDP.generate_scenario(scenario, ego_v, hit_point)
            if algorithm == "EmergancyBrakingSystem"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
            elseif algorithm == "PedestrianAvoidancePOMDP"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy, algorithm)
            elseif algorithm == "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, belief, action_pomdp, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = PedestrianAvoidancePOMDP.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, policy, algorithm)
            else
                println("No valid algorithm defined!")
                return false
            end
            
            (collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = PedestrianAvoidancePOMDP.evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision)
            println(collision, " ", emergency_brake_intervention, " ", dv_collision, " ", v_mean, " ", a_mean, " ", a_jerk, " ", a_min)
            result = [scenario_id, ego_v, hit_point, collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min  ]
            push!(results, result)
        end
    end 
end  


# store results from scneario evaluation in log file
df = DataFrame(results)
df = DataFrame(Matrix(df)')
rename!(df, :x1 => :scenario_id, :x2 => :ego_v, :x3 => :hit_point, :x4 => :collision, :x5 => :eb_intervention)
rename!(df, :x6 => :dv_collision, :x7 => :v_mean, :x8 => :a_mean, :x9 => :a_jerk, :x10 => :a_min)

CSV.write(log_filename, df);
df


results_EmergancyBrakingSystem_no.csv
Algorithm to evaluate: EmergancyBrakingSystem Policy: no (if specified)
true true 6.638888888888889 13.006076388888905 -9.117647058823529 10.0 -10.0
true true 1.1388888888888893 11.538194444444462 -9.464285714285714 10.0 -10.0
true true 1.1388888888888893 11.538194444444462 -9.464285714285714 10.0 -10.0
true true 1.1388888888888893 11.538194444444462 -9.464285714285714 10.0 -10.0


,scenario_id,ego_v,hit_point,collision,eb_intervention,dv_collision,v_mean,a_mean,a_jerk,a_min
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,13.8889,0.0,1.0,1.0,6.63889,13.0061,-9.11765,10.0,-10.0
2,2.0,13.8889,0.0,1.0,1.0,1.13889,11.5382,-9.46429,10.0,-10.0
3,3.0,13.8889,0.0,1.0,1.0,1.13889,11.5382,-9.46429,10.0,-10.0
4,4.0,13.8889,0.0,1.0,1.0,1.13889,11.5382,-9.46429,10.0,-10.0
